In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, auc, roc_curve

from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.initializers import Constant
from keras.layers import Dense, LSTM, Bidirectional
from keras.callbacks import ModelCheckpoint, EarlyStopping

from Libs.config import inter_extra_data_folder
from Libs.load_data import DataLoader, get_dataset_split
from Libs.keras_f1score import f1_m

2023-03-21 22:45:37.320421: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-21 22:45:37.463776: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-21 22:45:37.463813: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-21 22:45:38.278955: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
F_std = False

# initialize data loader
data_loader = DataLoader(run=30, N=1000, s=0.5, t=[0.01, 0.1, 0.5, 3], d=0.2, m=1, 
                         override=False, folder=inter_extra_data_folder)
# get the grid
grid_X, grid_y = data_loader.get_grid()
# get params dictionary
params = data_loader.get_params()

grid_X.shape, grid_y.shape

((30, 1, 4, 1, 1, 1000), (30, 1, 4, 1, 1, 1000))

A biggest assumption when training ANNs is the following: 

"We assume that training sets and test sets contains independent and identically distributed samples from the same unknown distribution $p_{data}(x,y)$"

This is a very important assumption that in general affect the performance ANNs, in particular classifier ones. We could, indeed, explore what can happen if we violete the following assumption. This a relevant application case, for exaple in cases when the generation parameters are not known.

In [3]:
def get_data_set(data1, mode, data2=None, F_std=False, overlap_size=15):
    assert mode in ['all', 'interpolation', 'extrapolation']    

    # params commons
    dataset_split_params = {
        'window_size': 20, # how large is the window
        'overlap_size': overlap_size, # how many time interval of overlap there is between the windows
        'label_treshold': 1, # how many labels have to be at 1 in the window_size to consider the current window as a flare
        'split_on_run': True, # if True the windows of a run cannot be on different sets
        'shuffle_run': False, # if True shuffles the order of the runs before computing the windows
        'shuffle_window': False, # if True shuffles the order of the windows in the resulting dataframes
        'test_size': 0.3, # size of the test set expressed in percentage
        'val_size': 0.2, # size of the validation set expressed in percentage, considered only if get_validation is True
        'get_validation': True, # if True the output would be train,val,test set, otherwise it would be train,test
        'random_state': 42 # sets the seed for reproducibility
    }

    if mode in ['interpolation', 'extrapolation']:
        assert not data2 is None
        grid_X_train, grid_y_train = data1
        grid_X_test, grid_y_test   = data2
        # get the train and validation set, selecting the index for grid given the interpolation assuption
        # notice that theta is the third parameter
        df_train, df_val, _ = get_dataset_split(grid_X_train, grid_y_train, **dataset_split_params)
        # get the test set, selecting the index for grid given the interpolation assuption
        # notice that theta is the third parameter
        _, _, df_test = get_dataset_split(grid_X_test, grid_y_test, **dataset_split_params)
    elif mode in ['all']:
        grid_X, grid_y = data1
        # get all the dataset from a single list
        df_train, df_val, df_test = get_dataset_split(grid_X, grid_y, **dataset_split_params)
    
    # number of classes
    print('Training set:')
    train_counts = df_train['future_flare'].value_counts()
    print(train_counts, '\n')
    print('validation set:')
    val_counts = df_val['future_flare'].value_counts()
    print(val_counts, '\n')
    print('Test set:')
    test_counts = df_test['future_flare'].value_counts()
    print(test_counts, '\n')
    print('Total:')
    total_counts = train_counts.add(val_counts).add(test_counts)
    print(total_counts, '\n')
    print()
    
    # compute the initial bias to pass then to the model
    initial_bias = Constant([np.log(train_counts[0]/train_counts[1])])

    # check the shape
    X_train, y_train = df_train.iloc[:,:-1].to_numpy(), df_train.future_flare.to_numpy()
    X_val, y_val = df_val.iloc[:,:-1].to_numpy(), df_val.future_flare.to_numpy()
    X_test, y_test = df_test.iloc[:,:-1].to_numpy(), df_test.future_flare.to_numpy()
    X = np.vstack((X_train, X_val, X_test))
    y = np.hstack((y_train, y_val, y_test))
    print('X ## Train:', X_train.shape, 'Val:', X_val.shape, 'Test:', X_test.shape)
    print('y ## Train:', y_train.shape, 'Val:', y_val.shape, 'Test:', y_test.shape)

    # finally, if requested, standardize the dataset
    if F_std:
        # Standardize Data
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train_std = scaler.transform(X_train)
        X_val_std = scaler.transform(X_val)
        X_test_std = scaler.transform(X_test)
        # get automatically the number of classes
        num_classes = len(np.unique(y))
    else:
        X_train_std = X_train
        X_val_std = X_val
        X_test_std = X_test


    # finally return the dataset
    return X_train_std, y_train, X_val_std, y_val, X_test_std, y_test, initial_bias

In [4]:
def make_model(X_train, initial_bias):
    # define model
    model = Sequential()
    model.add(Bidirectional(LSTM(20, activation='relu'), input_shape=(X_train.shape[1], 1)))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid',bias_initializer=initial_bias))
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=[f1_m, 'accuracy'])
    # print the summury model
    print(model.summary())
    return model

In [5]:
def eval(model, X_val, y_val, X_test, y_test):
    # Validation set
    y_pred = np.round(model.predict(X_val), 0)
    print("### Evaluation on validation set ###")
    print("Accuracy: %.2f" % (accuracy_score(y_pred, y_val)))
    print("F1 score: %.2f" % (f1_score(y_pred, y_val, average='macro')))
    fpr, tpr, _ = roc_curve(y_val, y_pred, pos_label=1)
    print('AUC:', auc(fpr, tpr))
    #Create confusion matrix and normalizes it over predicted (columns)
    cm = confusion_matrix(y_val, y_pred)
    print(cm)

    print()
    
    # Test set
    y_pred = np.round(model.predict(X_test), 0)
    print("### Evaluation on test set ###")
    print("Accuracy: %.2f" % (accuracy_score(y_pred, y_test)))
    print("F1 score: %.2f" % (f1_score(y_pred, y_test, average='macro')))
    fpr, tpr, _ = roc_curve(y_test, y_pred, pos_label=1)
    print('AUC:', auc(fpr, tpr))
    #Create confusion matrix and normalizes it over predicted (columns)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)

# LSTM model with multiple all theta parameters

Let's construct now the model

In [6]:
X_train, y_train, X_val, y_val, X_test, y_test, initial_bias = get_data_set((grid_X, grid_y), 'all')

Training set:
0    7197
1    3779
Name: future_flare, dtype: int64 

validation set:
0    3564
1    1924
Name: future_flare, dtype: int64 

Test set:
0    4547
1    2509
Name: future_flare, dtype: int64 

Total:
0    15308
1     8212
Name: future_flare, dtype: int64 


X ## Train: (10976, 20) Val: (5488, 20) Test: (7056, 20)
y ## Train: (10976,) Val: (5488,) Test: (7056,)


In [7]:
model = make_model(X_train, initial_bias)

2023-03-21 22:45:39.633675: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-21 22:45:39.633733: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-21 22:45:39.633754: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (44910f15382a): /proc/driver/nvidia/version does not exist
2023-03-21 22:45:39.634009: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 40)               3520      
 l)                                                              
                                                                 
 dense (Dense)               (None, 30)                1230      
                                                                 
 dense_1 (Dense)             (None, 10)                310       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 5,071
Trainable params: 5,071
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
epochs = 20
batch_size = 32
# define callbacks
callbacks = [
    ModelCheckpoint(
        os.path.join("models", "LSTM_allTheta_checkpoint.h5"), save_weights_only=True, monitor="val_loss"
    ),
    EarlyStopping(monitor="val_loss", patience=3, verbose=1),
]
# fit model
model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(X_val, y_val),
    verbose=1,
)

Epoch 1/20
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
343/343 [==============================] - 6s 12ms/step - loss: 0.3836 - f1_m: 0.7206 - accuracy: 0.7792 - val_loss: 0.2857 - val_f1_m: 0.6372 - val_accuracy: 0.8613
Epoch 2/20
343/343 [==============================] - 3s 10ms/step - loss: 0.2702 - f1_m: 0.7984 - accuracy: 0.8727 - val_loss: 0.2525 - val_f1_m: 0.6425 - val_accuracy: 0.8885
Epoch 3/20
343/343 [==============================] - 3s 10ms/step - loss: 0.2554 - f1_m: 0.8202 - accuracy: 0.8852 - val_loss: 0.2434 - val_f1_m: 0.6553 - val_accuracy: 0.8954
Epoch 4/20
343/343 [==============================] - 3s 10ms/step - loss: 0.2416 - f1_m: 0.8346 - accuracy: 0.8960 - val_loss: 0.2261 - val_f1_m: 0.6578 - val_accuracy: 0.9011
Epoch 5/20
343/343 [==============================] - 4s 10ms/step - loss: 0.2336 - f1_m: 0

Validation set results

In [9]:
eval(model, X_val, y_val, X_test, y_test)

172/172 [==============================] - 1s 4ms/step
### Evaluation on validation set ###
Accuracy: 0.93
F1 score: 0.92
AUC: 0.9049772383105716
[[3485   79]
 [ 323 1601]]

221/221 [==============================] - 1s 4ms/step
### Evaluation on test set ###
Accuracy: 0.92
F1 score: 0.91
AUC: 0.8966187964804601
[[4426  121]
 [ 452 2057]]


# Interpolation

Same model as before, but now we are fitting only in the dataset coming from the generation with using just the extreme parameters: 

$\theta=0.01$ and $\theta=3$

and a fraction of the other dataset, coming from $\theta=0.1$ and $\theta=0.5$ as test set

In [10]:
p = 'theta'
# train configurations
theta_train_list     = [0.01, 3]
theta_train_list_idx = [params[p].index(t) for t in theta_train_list]
data_train = (grid_X[:,:,theta_train_list_idx,:,:,:], grid_y[:,:,theta_train_list_idx,:,:,:])
# test configuration
theta_test_list      = [0.1, 0.5]
theta_test_list_idx  = [params[p].index(t) for t in theta_test_list]
data_test = (grid_X[:,:,theta_test_list_idx,:,:,:], grid_y[:,:,theta_test_list_idx,:,:,:])

X_train, y_train, X_val, y_val, X_test, y_test, initial_bias = get_data_set(data_train, 'interpolation', data2=data_test)

Training set:
0    3903
1    1585
Name: future_flare, dtype: int64 

validation set:
0    1975
1     769
Name: future_flare, dtype: int64 

Test set:
0    2042
1    1486
Name: future_flare, dtype: int64 

Total:
0    7920
1    3840
Name: future_flare, dtype: int64 


X ## Train: (5488, 20) Val: (2744, 20) Test: (3528, 20)
y ## Train: (5488,) Val: (2744,) Test: (3528,)


In [11]:
model = make_model(X_train, initial_bias)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_1 (Bidirectio  (None, 40)               3520      
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 30)                1230      
                                                                 
 dense_4 (Dense)             (None, 10)                310       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 5,071
Trainable params: 5,071
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
epochs = 20
batch_size = 32
# define callbacks
callbacks = [
    ModelCheckpoint(
        os.path.join("models", "LSTM_intrpTheta_checkpoint.h5"), save_weights_only=True, monitor="val_loss"
    ),
    EarlyStopping(monitor="val_loss", patience=3, verbose=1),
]
# fit model
model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(X_val, y_val),
    verbose=1,
)

Epoch 1/20
172/172 [==============================] - 5s 15ms/step - loss: 0.5598 - f1_m: 0.4603 - accuracy: 0.6279 - val_loss: 0.3139 - val_f1_m: 0.4448 - val_accuracy: 0.8509
Epoch 2/20
172/172 [==============================] - 2s 13ms/step - loss: 0.3128 - f1_m: 0.6854 - accuracy: 0.8375 - val_loss: 0.2985 - val_f1_m: 0.4466 - val_accuracy: 0.8528
Epoch 3/20
172/172 [==============================] - 2s 13ms/step - loss: 0.3125 - f1_m: 0.6910 - accuracy: 0.8373 - val_loss: 0.2990 - val_f1_m: 0.4553 - val_accuracy: 0.8560
Epoch 4/20
172/172 [==============================] - 2s 14ms/step - loss: 0.3037 - f1_m: 0.6987 - accuracy: 0.8437 - val_loss: 0.3033 - val_f1_m: 0.3970 - val_accuracy: 0.8440
Epoch 5/20
172/172 [==============================] - 2s 13ms/step - loss: 0.3004 - f1_m: 0.7170 - accuracy: 0.8499 - val_loss: 0.2949 - val_f1_m: 0.4095 - val_accuracy: 0.8477
Epoch 6/20
172/172 [==============================] - 2s 13ms/step - loss: 0.2951 - f1_m: 0.7114 - accuracy: 0.8478

In [13]:
eval(model, X_val, y_val, X_test, y_test)

86/86 [==============================] - 1s 4ms/step
### Evaluation on validation set ###
Accuracy: 0.88
F1 score: 0.85
AUC: 0.8452489670951918
[[1844  131]
 [ 187  582]]

111/111 [==============================] - 0s 4ms/step
### Evaluation on test set ###
Accuracy: 0.91
F1 score: 0.90
AUC: 0.8949786647297728
[[1969   73]
 [ 259 1227]]


# Extrapolation

Same model as before, but now we are fitting only in the dataset coming from the generation without using the extreme parameters: 

$\theta=0.1$ and $\theta=0.5$

In [14]:
p = 'theta'
# train configurations
theta_train_list     = [0.1, 0.5]
theta_train_list_idx = [params[p].index(t) for t in theta_train_list]
data_train = (grid_X[:,:,theta_train_list_idx,:,:,:], grid_y[:,:,theta_train_list_idx,:,:,:])
# test configuration
theta_test_list      = [0.01, 3]
theta_test_list_idx  = [params[p].index(t) for t in theta_test_list]
data_test = (grid_X[:,:,theta_test_list_idx,:,:,:], grid_y[:,:,theta_test_list_idx,:,:,:])

X_train, y_train, X_val, y_val, X_test, y_test, initial_bias = get_data_set(data_train, 'extrapolation', data2=data_test)

Training set:
0    3294
1    2194
Name: future_flare, dtype: int64 

validation set:
0    1589
1    1155
Name: future_flare, dtype: int64 

Test set:
0    2505
1    1023
Name: future_flare, dtype: int64 

Total:
0    7388
1    4372
Name: future_flare, dtype: int64 


X ## Train: (5488, 20) Val: (2744, 20) Test: (3528, 20)
y ## Train: (5488,) Val: (2744,) Test: (3528,)


In [15]:
model = make_model(X_train, initial_bias)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 40)               3520      
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 30)                1230      
                                                                 
 dense_7 (Dense)             (None, 10)                310       
                                                                 
 dense_8 (Dense)             (None, 1)                 11        
                                                                 
Total params: 5,071
Trainable params: 5,071
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
epochs = 20
batch_size = 32
# define callbacks
callbacks = [
    ModelCheckpoint(
        os.path.join("models", "LSTM_extrpTheta_checkpoint.h5"), save_weights_only=True, monitor="val_loss"
    ),
    EarlyStopping(monitor="val_loss", patience=3, verbose=1),
]
# fit model
model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(X_val, y_val),
    verbose=1,
)

Epoch 1/20
172/172 [==============================] - 5s 14ms/step - loss: 0.4835 - f1_m: 0.7270 - accuracy: 0.7026 - val_loss: 0.2695 - val_f1_m: 0.8131 - val_accuracy: 0.8885
Epoch 2/20
172/172 [==============================] - 2s 13ms/step - loss: 0.2562 - f1_m: 0.8518 - accuracy: 0.8905 - val_loss: 0.2570 - val_f1_m: 0.8102 - val_accuracy: 0.8907
Epoch 3/20
172/172 [==============================] - 2s 13ms/step - loss: 0.2419 - f1_m: 0.8626 - accuracy: 0.8981 - val_loss: 0.2986 - val_f1_m: 0.7652 - val_accuracy: 0.8739
Epoch 4/20
172/172 [==============================] - 2s 13ms/step - loss: 0.2242 - f1_m: 0.8722 - accuracy: 0.9045 - val_loss: 0.2312 - val_f1_m: 0.8321 - val_accuracy: 0.8983
Epoch 5/20
172/172 [==============================] - 2s 13ms/step - loss: 0.2114 - f1_m: 0.8798 - accuracy: 0.9140 - val_loss: 0.2068 - val_f1_m: 0.8560 - val_accuracy: 0.9103
Epoch 6/20
172/172 [==============================] - 2s 13ms/step - loss: 0.2106 - f1_m: 0.8862 - accuracy: 0.9136

In [17]:
eval(model, X_val, y_val, X_test, y_test)

86/86 [==============================] - 1s 5ms/step
### Evaluation on validation set ###
Accuracy: 0.92
F1 score: 0.92
AUC: 0.912090699315369
[[1538   51]
 [ 166  989]]

111/111 [==============================] - 1s 5ms/step
### Evaluation on test set ###
Accuracy: 0.89
F1 score: 0.86
AUC: 0.8506365567984265
[[2386  119]
 [ 257  766]]


## Further investigation in extrapolation with greater overlap

In [18]:
X_train, y_train, X_val, y_val, X_test, y_test, initial_bias = get_data_set(data_train, 'extrapolation', data2=data_test, overlap_size=19)

Training set:
0    16438
1    11002
Name: future_flare, dtype: int64 

validation set:
0    7930
1    5790
Name: future_flare, dtype: int64 

Test set:
0    12523
1     5117
Name: future_flare, dtype: int64 

Total:
0    36891
1    21909
Name: future_flare, dtype: int64 


X ## Train: (27440, 20) Val: (13720, 20) Test: (17640, 20)
y ## Train: (27440,) Val: (13720,) Test: (17640,)


In [19]:
model = make_model(X_train, initial_bias)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_3 (Bidirectio  (None, 40)               3520      
 nal)                                                            
                                                                 
 dense_9 (Dense)             (None, 30)                1230      
                                                                 
 dense_10 (Dense)            (None, 10)                310       
                                                                 
 dense_11 (Dense)            (None, 1)                 11        
                                                                 
Total params: 5,071
Trainable params: 5,071
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
epochs = 20
batch_size = 32
# define callbacks
callbacks = [
    ModelCheckpoint(
        os.path.join("models", "LSTM_extrpTheta19_checkpoint.h5"), save_weights_only=True, monitor="val_loss"
    ),
    EarlyStopping(monitor="val_loss", patience=3, verbose=1),
]
# fit model
model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(X_val, y_val),
    verbose=1,
)

Epoch 1/20
858/858 [==============================] - 18s 18ms/step - loss: 0.2378 - f1_m: 0.8629 - accuracy: 0.8778 - val_loss: 0.1487 - val_f1_m: 0.6013 - val_accuracy: 0.9361
Epoch 2/20
858/858 [==============================] - 17s 19ms/step - loss: 0.1422 - f1_m: 0.9197 - accuracy: 0.9388 - val_loss: 0.1166 - val_f1_m: 0.6161 - val_accuracy: 0.9489
Epoch 3/20
858/858 [==============================] - 16s 19ms/step - loss: 0.1168 - f1_m: 0.9349 - accuracy: 0.9502 - val_loss: 0.0984 - val_f1_m: 0.6207 - val_accuracy: 0.9614
Epoch 4/20
858/858 [==============================] - 16s 19ms/step - loss: 0.1020 - f1_m: 0.9442 - accuracy: 0.9570 - val_loss: 0.1214 - val_f1_m: 0.6301 - val_accuracy: 0.9483
Epoch 5/20
858/858 [==============================] - 16s 19ms/step - loss: 0.0921 - f1_m: 0.9487 - accuracy: 0.9611 - val_loss: 0.0950 - val_f1_m: 0.6274 - val_accuracy: 0.9658
Epoch 6/20
858/858 [==============================] - 16s 19ms/step - loss: 0.0845 - f1_m: 0.9555 - accuracy: 

In [21]:
eval(model, X_val, y_val, X_test, y_test)

429/429 [==============================] - 2s 4ms/step
### Evaluation on validation set ###
Accuracy: 0.98
F1 score: 0.98
AUC: 0.9810361387529484
[[7847   83]
 [ 159 5631]]

552/552 [==============================] - 2s 4ms/step
### Evaluation on test set ###
Accuracy: 0.98
F1 score: 0.97
AUC: 0.9716514499777318
[[12393   130]
 [  237  4880]]
